In [1]:
# Library Imports
import pickle
import streamlit as st # Version 1.25.0
import pandas as pd # Version 2.0.1
import numpy as np # Version 1.3.0
import seaborn as sns # Version 0.12.2
import matplotlib.pyplot as plt # version 3.6.3
from sklearn.ensemble import GradientBoostingRegressor # Version 1.3.0

In [2]:
data1 = pd.read_csv("../Data/raw_combined.csv.gz", compression="gzip")

data1

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,52582829,https://www.airbnb.com/rooms/52582829,20220615002624,2022-06-15,Huge Oceanview decks+RooftopDeck☀sleeps 10☀Gar...,10 steps from the boardwalk! Beautiful beach h...,NaN,https://a0.muscache.com/pictures/ccfcc669-b8ff...,263534443,https://www.airbnb.com/users/show/263534443,...,4.77,4.90,4.77,NaN,t,30,30,0,0,3.77
1,53085803,https://www.airbnb.com/rooms/53085803,20220615002624,2022-06-15,Sunny 1 bedroom home next to pool & tennis club,"Spacious sunny, 1 bedroom home centrally locat...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,10469115,https://www.airbnb.com/users/show/10469115,...,5.00,4.90,4.80,NaN,t,2,1,1,0,3.41
2,53082401,https://www.airbnb.com/rooms/53082401,20220615002624,2022-06-15,Casita on the Park,Private studio in lovely Marston Hills. Walkin...,NaN,https://a0.muscache.com/pictures/5d97009a-06f3...,100963539,https://www.airbnb.com/users/show/100963539,...,5.00,5.00,4.88,NaN,t,1,1,0,0,2.96
3,53010441,https://www.airbnb.com/rooms/53010441,20220615002624,2022-06-15,Private Room/King Bed,Private bedroom with a private full bath in a ...,NaN,https://a0.muscache.com/pictures/96cc043b-cff9...,14064859,https://www.airbnb.com/users/show/14064859,...,5.00,5.00,5.00,NaN,t,10,5,5,0,0.14
4,53011657,https://www.airbnb.com/rooms/53011657,20220615002624,2022-06-15,Quiet Bedroom in the heart of Pacific Beach,"Enjoy staying in the heart of Pacific Beach, S...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,145032731,https://www.airbnb.com/users/show/145032731,...,5.00,5.00,5.00,NaN,t,1,0,1,0,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18622,36210645,https://www.airbnb.com/rooms/36210645,20230324234210,2023-03-25,"Hotel Z, Z Queen",You’ll get your much needed pampering experien...,NaN,https://a0.muscache.com/pictures/5c044388-31be...,270119516,https://www.airbnb.com/users/show/270119516,...,4.97,4.94,4.76,Exempt,t,14,0,14,0,1.78
18623,36210735,https://www.airbnb.com/rooms/36210735,20230324234210,2023-03-25,"Hotel Z, Z Queen w/Comm. Features",You’ll get your much needed pampering experien...,NaN,https://a0.muscache.com/pictures/08554730-4577...,270119516,https://www.airbnb.com/users/show/270119516,...,4.93,4.63,4.63,Exempt,t,14,0,14,0,0.79
18624,35768403,https://www.airbnb.com/rooms/35768403,20230324234210,2023-03-25,"Del Mar Area, pool, beaches, restaurants, shop...",Beautiful immaculate remodeled home. <br />Per...,Walk to the end of the block to a grass and fl...,https://a0.muscache.com/pictures/80f55111-3294...,267506060,https://www.airbnb.com/users/show/267506060,...,4.68,4.97,4.68,NaN,f,1,1,0,0,0.85
18625,36209188,https://www.airbnb.com/rooms/36209188,20230324234210,2023-03-25,Sorrento Valley Haven,Enjoy this spacious home with the private gard...,The location is best of all worlds. Quiet nei...,https://a0.muscache.com/pictures/ff9a9c10-60df...,10348536,https://www.airbnb.com/users/show/10348536,...,5.00,5.00,5.00,NaN,f,2,1,1,0,0.13


In [11]:
import re
url_match = re.compile(r"[\w\W]*\?")

url = re.match(r"[\w\W]*\?", """
https://www.airbnb.com/rooms/50228360?adults=1&category_tag=
Tag%3A7769&children=0&enable_m3_private_room=true&infants=0&pets=
0&search_mode=flex_destinations_search&check_in=2023-09-07&check_out=
2023-09-14&source_impression_id=p3_1690775026_EM%2FlsHtKrNWan0kZ&previous_page
_section_name=1000&federated_search_id=9deded69-3a20-4f10-b22a-474c3f4ecc1b
""").group(0).strip()[:-1]

In [13]:
data1.loc[data1["listing_url"] == url]

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
2328,50228360,https://www.airbnb.com/rooms/50228360,20220918143850,2022-09-18,2 Bd Poolside Condo in Beachfront Community!,"Welcome to the beautiful Del Mar Beach Club, i...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,50931016,https://www.airbnb.com/users/show/50931016,...,4.72,4.92,4.72,NaN,f,3,3,0,0,1.64


In [4]:
# Read in Data
data = pd.read_csv("../Data/model_ready.csv.gz", compression = "gzip")

# Load models
house_model = pickle.load(open("../Code Library/Models/gb_house_tuned.sav", "rb"))
room_model = pickle.load(open("../Code Library/Models/gb_room_tuned.sav", "rb"))

# Load fitted pipelines
house_pipeline = pickle.load(open("../Deployment_Files/house_pipeline.sav", "rb"))
room_pipeline = pickle.load(open("../Deployment_Files/room_pipeline.sav", "rb"))

In [5]:
data

,host_listings_count,property_type,room_type,bathrooms,bedrooms,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_maximum_nights,...,calculated_host_listings_count,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,zipcode,median_income_dollars,property_type_binary,private,sentiment,review_score_weighted
0,52.0,Entire home,Entire home/apt,2.0,2.0,728.0,2,28,1.0,1125.0,...,30,0,0,3.77,92109,95170.0,house,1,0.900577,150.04
1,0.0,Private room in condo,Private room,1.0,1.0,70.0,7,30,7.0,30.0,...,1,1,0,0.15,92109,95170.0,room,1,0.997769,5.00
2,4.0,Entire home,Entire home/apt,3.0,4.0,595.0,3,365,3.0,365.0,...,3,0,0,1.53,92109,95170.0,house,1,0.999661,45.00
3,0.0,Entire home,Entire home/apt,1.0,3.0,926.0,2,14,2.0,1125.0,...,2,0,0,0.00,92109,95170.0,house,1,0.999559,0.00
4,4.0,Camper/RV,Entire home/apt,1.0,1.0,62.0,1,1125,1.0,1125.0,...,3,1,0,0.12,92109,95170.0,house,0,0.062897,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18489,1.0,Entire rental unit,Entire home/apt,1.0,1.0,96.0,4,1125,4.0,1125.0,...,1,0,0,2.30,92020,64691.0,house,1,0.321106,748.50
18490,4.0,Entire home,Entire home/apt,2.0,3.0,262.0,1,365,1.0,365.0,...,3,0,0,1.00,92020,64691.0,house,1,0.844013,5.00
18491,1.0,Private room in home,Private room,1.0,1.0,50.0,2,30,2.0,30.0,...,1,1,0,0.53,92020,64691.0,room,1,0.999578,156.16
18492,1.0,Entire guesthouse,Entire home/apt,1.0,2.0,175.0,1,365,1.0,365.0,...,1,0,0,4.23,92020,64691.0,house,1,0.057779,50.00


In [72]:
data[["host_listings_count", "calculated_host_listings_count"]].head(50)

,host_listings_count,calculated_host_listings_count
0,52.0,30
1,0.0,1
2,4.0,3
3,0.0,2
4,4.0,3
5,3.0,2
6,5.0,5
7,0.0,2
8,3.0,2
9,8.0,8


In [40]:
test_dict = data.set_index("zipcode").to_dict()["median_income_dollars"]
test_dict[92127]

156464.0

In [70]:
data.loc[(data["zipcode"] == int(92127)) 
        & (data["property_type_binary"] == "house")]["instant_bookable"].mode()

0    f
Name: instant_bookable, dtype: object

In [18]:
type = "Entire home/apt"
property_type_binary_feat21 = "room" if "room" in type.lower() else "house"
property_type_binary_feat21

'house'